In [ ]:
import pandas as pd
from utils import add_prefix_to_colnames, percentile, onehot, summary_group_cate_data, summary_group_num_data

# Create after_trans_payment_for_loan.csv
For cost calculation

In [2]:
loan_df = pd.read_csv('data/loan.csv')
trans_before_loan = pd.read_csv('data/trans_before_loan.csv')
trans_after_loan = pd.read_csv('data/trans_after_loan.csv')

Find out that Monthly transaction of after transactin, when stop to pay loan.

In [ ]:
trans_df_in = trans_after_loan[trans_after_loan['k_symbol'].notna()]
trans_df_in.head()

In [ ]:
trans_df_ = trans_df_in[trans_df_in['k_symbol'] == 'loan'].sort_values(by=['account_id', 'date']).copy()
trans_df_ = trans_df_.groupby(by='account_id')[['k_symbol']].count().reset_index()
trans_df_ = trans_df_.rename(columns={'k_symbol':'count_loan_trans'})
trans_df_

In [ ]:
df = pd.merge(loan_df, trans_df_, how='left', on=['account_id'])
df.head()

In [ ]:
df.loc[df['status']=='C', ['count_loan_trans']] = df['duration']
df.head()

In [ ]:
t = df.loc[df['status']=='B']
# t['num_month_paid'] = t['duration'] - t['count_loan_trans']
t['%num_month_paid'] = t['count_loan_trans'].div( t['duration'], axis=0) * 100
t['%num_month_paid'].mean()

assume that D paid for 80 percent of total loan amount

In [ ]:
df.loc[df['status']=='D', ['count_loan_trans']] = (df['duration'] * 80/100).round()
df = df[['account_id', 'count_loan_trans']]
df

In [ ]:
df.to_csv('after_trans_payment_for_loan.csv', index=False)

# Profit calculation

In [8]:
import numpy as np
import pandas as pd
from profit import cal_cost, cal_revenue

In [7]:
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')

def profit_thresholding(df, th_, profit, interest_rate):
    df['y_pred'] = to_labels(df['probs'], th_)
    # TN
    df.loc[(df['y_true']==0) & (df['y_pred']==0), ['profit']] = profit
    df.loc[(df['y_true']==0) & (df['y_pred']==0), ['conf']] = 'tn'

    # TP 
    df.loc[(df['y_true']==1) & (df['y_pred']==1), ['profit']] = 0
    df.loc[(df['y_true']==1) & (df['y_pred']==1), ['conf']] = 'tp'

    # FN
    df.loc[(df['y_true']==0) & (df['y_pred']==1), ['profit']] = 0
    df.loc[(df['y_true']==0) & (df['y_pred']==1), ['conf']] = 'fn'
    
    # FP
    df.loc[(df['y_true']==1) & (df['y_pred']==0), ['profit']] = profit
    df.loc[(df['y_true']==1) & (df['y_pred']==0), ['conf']] = 'fp'
            
    df['threshold'] = th_
    df['interest_rate'] = interest_rate
    
    return df

In [6]:
report = pd.read_csv('report_xgb.csv')
report['y_pred'] = np.nan
report['case_TP'] = np.nan
report['case_FN'] = np.nan
report['case_FP'] = np.nan
report['case_TN'] = np.nan
report['threshold'] = np.nan
report

,y_true,probs,account_id,y_pred,case_TP,case_FN,case_FP,case_TN,threshold
0,0.0,0.005088,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0.349875,19.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.003227,25.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,0.994965,37.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.002298,38.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
677,0.0,0.010875,11327.0,NaN,NaN,NaN,NaN,NaN,NaN
678,0.0,0.003006,11328.0,NaN,NaN,NaN,NaN,NaN,NaN
679,0.0,0.045928,11349.0,NaN,NaN,NaN,NaN,NaN,NaN
680,0.0,0.006072,11359.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
loan_df = pd.read_csv('data/loan.csv')
loan_df.head()

,loan_id,account_id,date,amount,duration,payments,status
0,4959,2,1994-01-05,80952,24,3373.0,A
1,4961,19,1996-04-29,30276,12,2523.0,B
2,4962,25,1997-12-08,30276,12,2523.0,A
3,4967,37,1998-10-14,318480,60,5308.0,D
4,4968,38,1998-04-19,110736,48,2307.0,C


In [ ]:
filename = 'after_trans_payment_for_loan.csv'
after_trans_for_loan = pd.read_csv(filename)

interest_rate_peryear = [0.01, 0.03, 0.05, 0.1, 0.2]
profits = []
report = pd.read_csv('report_xgb.csv')
reports = []

for i in interest_rate_peryear:
    revenue = cal_revenue(total_amount=loan_df['amount'], 
                    loan_duration_month=loan_df['duration'], 
                    paid_month=after_trans_for_loan['count_loan_trans'], 
                    interest_rate_peryear=i)

    cost = cal_cost(loan_payment=loan_df['payments'], 
                    loan_duration_month=loan_df['duration'],
                    paid_month=after_trans_for_loan['count_loan_trans'])

    profit = revenue - cost



    
    
    for th in np.arange(0, 1.1, 0.1):
        th_ = th.round(decimals=1)
        r = profit_thresholding(report, th_, profit, i).reset_index(drop=True)
        reports.append(r)
          
    profits.append(profit.sum())

reports = pd.concat(reports)
reports